# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import ast
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import re
from fuzzywuzzy import process
import dask.dataframe as dd
import os
import sys

# CSVs Loading

In [2]:
# Load the DataFrame
reviews = pd.read_csv('/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/reviews_emotions.csv')

In [3]:
reviews.head()

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,predicted_moods
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,"[('excitement', 35.543839830098335), ('approva..."
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,"[('disapproval', 56.05641547237329), ('annoyan..."
2,city_hunter_shinjuku_private_eyes,City Hunter: Shinjuku Private Eyes,2590987,2019-05-28,Reuben Baron,False,fresh,CBR,The choreography is so precise and lifelike at...,POSITIVE,The choreography is so precise and lifelike at...,"[('admiration', 80.08288365216127), ('approval..."
3,city_hunter_shinjuku_private_eyes,City Hunter: Shinjuku Private Eyes,2558908,2019-02-14,Matt Schley,False,rotten,Japan Times,The film's out-of-touch attempts at humor may ...,NEGATIVE,The films outoftouch attempts at humor may fin...,"[('amusement', 22.690977576957067), ('realizat..."
4,dangerous_men_2015,Dangerous Men,2504681,2018-08-29,Pat Padua,False,fresh,DCist,Its clumsy determination is endearing and some...,POSITIVE,Its clumsy determination is endearing and some...,"[('amusement', 49.197768434566136), ('admirati..."


In [4]:
movies = pd.read_csv('/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/clean_movies.csv')

In [5]:
movies.head()

,id,title,audienceScore,tomatoMeter,runtimeMinutes,genre,originalLanguage,director,writer,release_year
0,love_lies,"Love, Lies",43.0,65.76,120.0,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown
1,adrift_2018,Adrift,65.0,69.00,120.0,Adventure,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018
2,adrift_2018,Adrift,65.0,69.00,120.0,Drama,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018
3,adrift_2018,Adrift,65.0,69.00,120.0,Romance,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018
4,1035316-born_to_kill,Born to Kill,74.0,83.00,92.0,Crime,English,Robert Wise,"Eve Greene,Richard Macaulay",1947


# Data Wrangling

### Movies

Group by id and transform the genres into a list:

In [6]:
movies_grouped = movies.groupby('id')['genre'].apply(list).reset_index()

# Merge the result back into the original DataFrame, keeping all columns
movies = movies.drop(columns=['genre']).drop_duplicates(subset=['id']).merge(movies_grouped, on='id', how='left')

In [7]:
movies.head()

,id,title,audienceScore,tomatoMeter,runtimeMinutes,originalLanguage,director,writer,release_year,genre
0,love_lies,"Love, Lies",43.00,65.76,120.0,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,[Drama]
1,adrift_2018,Adrift,65.00,69.00,120.0,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018,"[Adventure, Drama, Romance]"
2,1035316-born_to_kill,Born to Kill,74.00,83.00,92.0,English,Robert Wise,"Eve Greene,Richard Macaulay",1947,"[Crime, Drama]"
3,garden_murder_case,The Garden Murder Case,55.67,65.76,61.0,English,Edwin L. Marin,Unknown,2016,[Thriller]
4,margarita_happy_hour,Margarita Happy Hour,55.67,76.00,98.0,English,Ilya Chaiken,Ilya Chaiken,2002,[Drama]


In [8]:
movies.loc[movies['title'] == 'How It Ends']

,id,title,audienceScore,tomatoMeter,runtimeMinutes,originalLanguage,director,writer,release_year,genre
13046,how_it_ends_2021,How It Ends,42.0,68.0,82.0,English,"Zoe Lister Jones,Daryl Wein","Daryl Wein,Zoe Lister Jones",2021,"[Comedy, Drama]"
47022,how_it_ends,How It Ends,17.0,17.0,113.0,English,David M. Rosenthal,Brooks McLaren,2018,"[Sci-fi, Thriller, Action, Adventure]"


### Reviews

#### Calculating the average emotions per movie

#### Explode and split the data
First, explode and split the predicted_moods column into emotion and probability:

In [9]:
# Convert tuple strings into actual tuples
reviews['predicted_moods'] = reviews['predicted_moods'].apply(ast.literal_eval)

# Explode the column
reviews_exploded = reviews.explode('predicted_moods')

# Split the tuples into separate columns
reviews_exploded[['emotion', 'probability']] = pd.DataFrame(reviews_exploded['predicted_moods'].tolist(), index=reviews_exploded.index)
reviews_exploded = reviews_exploded.drop(columns=['predicted_moods'])

# Convert 'probability' to numeric
reviews_exploded['probability'] = pd.to_numeric(reviews_exploded['probability'])

# Verify the result
reviews_exploded

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,emotion,probability
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,excitement,35.543840
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,approval,33.460199
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,admiration,14.496543
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,disapproval,56.056415
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,annoyance,16.554233
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,annoyance,29.755378
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,disapproval,16.977576
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,amusement,66.380384
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,joy,22.703058


#### Normalize probabilities to sum to 100%

In [10]:
# Normalize probabilities to sum to 100%
sum_probabilities = reviews_exploded.groupby('reviewId')['probability'].transform('sum')
reviews_exploded['probability_normalized'] = (reviews_exploded['probability'] / sum_probabilities) * 100

# Round the normalized probabilities to 4 decimal places
reviews_exploded['probability_normalized'] = reviews_exploded['probability_normalized'].round(4)

# Verify the result
reviews_exploded

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,emotion,probability,probability_normalized
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,excitement,35.543840,42.5672
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,approval,33.460199,40.0718
0,beavers,Beavers,1145982,2003-05-23,Ivan M. Lincoln,False,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,Timed to be just long enough for most youngste...,admiration,14.496543,17.3610
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,disapproval,56.056415,70.6446
1,blood_mask,Blood Mask,1636744,2007-06-02,The Foywonder,False,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,It doesnt matter if a movie costs 300 million ...,annoyance,16.554233,20.8623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,annoyance,29.755378,37.8302
1363577,thor_love_and_thunder,Thor: Love and Thunder,102706148,2022-07-05,Jake Cole,True,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,Across Taika Waititis film a war against the g...,disapproval,16.977576,21.5848
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,amusement,66.380384,71.3199
1363578,thor_love_and_thunder,Thor: Love and Thunder,102706147,2022-07-05,Roger Moore,False,fresh,Movie Nation,It&#8217;s the jokes that make it&#44; with th...,POSITIVE,Its the jokes that make it with the selfmockin...,joy,22.703058,24.3925


In [11]:
# Verify the sum of normalized probabilities
sum_normalized = reviews_exploded.groupby('reviewId')['probability_normalized'].sum()

# Round the sums to 2 decimal places for display
sum_normalized = sum_normalized.round(2)

# Check unique values
print(sum_normalized.unique())

[100.]


#### Group by id and emotion and calculate the mean probability

In [12]:
reviews_grouped = reviews_exploded.groupby(['id', 'emotion'])['probability_normalized'].mean().reset_index()

# Verify the result
reviews_grouped.head(10)

,id,emotion,probability_normalized
0,$5_a_day,admiration,43.15200
1,$5_a_day,amusement,33.83000
2,$5_a_day,annoyance,36.67500
3,$5_a_day,approval,34.83245
4,$5_a_day,disappointment,32.67010
5,$5_a_day,disapproval,50.40925
6,$5_a_day,joy,21.32340
7,$5_a_day,optimism,11.22480
8,$5_a_day,realization,7.48920
9,009_re_cyborg,admiration,49.29476


In [13]:
pd.set_option('display.max_colwidth', None)

reviews_exploded.loc[reviews_exploded['id'] == '$5_a_day']

,id,title,reviewId,creationDate,criticName,isTopCritic,reviewState,publicatioName,reviewText,scoreSentiment,cleanedReviewText,emotion,probability,probability_normalized
1213414,$5_a_day,Five Dollars a Day,2097498,2012-08-01,Kevin Carr,False,rotten,7M Pictures,"$5 a Day isn't perfect, but it does examine some of the issues that we have when connecting with our parents as adults.",NEGATIVE,5 a Day isnt perfect but it does examine some of the issues that we have when connecting with our parents as adults,disapproval,60.371897,70.1637
1213414,$5_a_day,Five Dollars a Day,2097498,2012-08-01,Kevin Carr,False,rotten,7M Pictures,"$5 a Day isn't perfect, but it does examine some of the issues that we have when connecting with our parents as adults.",NEGATIVE,5 a Day isnt perfect but it does examine some of the issues that we have when connecting with our parents as adults,approval,19.228371,22.3471
1213414,$5_a_day,Five Dollars a Day,2097498,2012-08-01,Kevin Carr,False,rotten,7M Pictures,"$5 a Day isn't perfect, but it does examine some of the issues that we have when connecting with our parents as adults.",NEGATIVE,5 a Day isnt perfect but it does examine some of the issues that we have when connecting with our parents as adults,realization,6.444050,7.4892
1213415,$5_a_day,Five Dollars a Day,1929215,2010-09-09,Brian Orndorf,False,rotten,DVDTalk.com,"Dreadfully formulaic and absent a thoughtful emotional core, the picture is best valued as a forgettable trifle starring Hollywood's most enduring weirdo.",NEGATIVE,Dreadfully formulaic and absent a thoughtful emotional core the picture is best valued as a forgettable trifle starring Hollywoods most enduring weirdo,annoyance,24.614549,36.6750
1213415,$5_a_day,Five Dollars a Day,1929215,2010-09-09,Brian Orndorf,False,rotten,DVDTalk.com,"Dreadfully formulaic and absent a thoughtful emotional core, the picture is best valued as a forgettable trifle starring Hollywood's most enduring weirdo.",NEGATIVE,Dreadfully formulaic and absent a thoughtful emotional core the picture is best valued as a forgettable trifle starring Hollywoods most enduring weirdo,disappointment,21.926653,32.6701
1213415,$5_a_day,Five Dollars a Day,1929215,2010-09-09,Brian Orndorf,False,rotten,DVDTalk.com,"Dreadfully formulaic and absent a thoughtful emotional core, the picture is best valued as a forgettable trifle starring Hollywood's most enduring weirdo.",NEGATIVE,Dreadfully formulaic and absent a thoughtful emotional core the picture is best valued as a forgettable trifle starring Hollywoods most enduring weirdo,disapproval,20.574082,30.6548
1213416,$5_a_day,Five Dollars a Day,1924503,2010-08-19,Jules Brenner,False,fresh,Cinema Signals,The success of the piece rests on Nivola's calm adaptability as fatherly hi-jinx tries his patience and makes its mark on his heart.,POSITIVE,The success of the piece rests on Nivolas calm adaptability as fatherly hijinx tries his patience and makes its mark on his heart,approval,37.784600,47.3178
1213416,$5_a_day,Five Dollars a Day,1924503,2010-08-19,Jules Brenner,False,fresh,Cinema Signals,The success of the piece rests on Nivola's calm adaptability as fatherly hi-jinx tries his patience and makes its mark on his heart.,POSITIVE,The success of the piece rests on Nivolas calm adaptability as fatherly hijinx tries his patience and makes its mark on his heart,admiration,33.104835,41.4574
1213416,$5_a_day,Five Dollars a Day,1924503,2010-08-19,Jules Brenner,False,fresh,Cinema Signals,The success of the piece rests on Nivola's calm adaptability as fatherly hi-jinx tries his patience and makes its mark on his heart.,POSITIVE,The success of the piece rests on Nivolas calm adaptability as fatherly hijinx tries his patience and makes its mark on his heart,optimism,8.963305,11.2248
1213417,$5_a_day,Five Dollars a Day,1780984,2008-11-16,David Nusair,False,fresh,Reel Film Reviews,...very amusing and agreeable...,POSITIVE,very amusing and agreeable,admiration,40.276130,44.8466


In [14]:
# Select the 3 emotions with the highest average per movie
top_3_emotions = reviews_grouped.groupby('id').apply(
    lambda x: x.nlargest(3, 'probability_normalized')
).reset_index(drop=True)

In [15]:
top_3_emotions

,id,emotion,probability_normalized
0,$5_a_day,disapproval,50.40925
1,$5_a_day,admiration,43.15200
2,$5_a_day,annoyance,36.67500
3,009_re_cyborg,disapproval,56.85230
4,009_re_cyborg,annoyance,50.76375
...,...,...,...
202807,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,approval,58.79150
202808,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,fear,33.92220
202809,zz_top_that_little_ol_band_from_texas,amusement,57.87560
202810,zz_top_that_little_ol_band_from_texas,joy,56.15785


In [16]:
# Normalize probabilities to sum 100% for each movie
top_3_emotions['probability_normalized'] = top_3_emotions.groupby('id')['probability_normalized'].transform(
    lambda x: (x / x.sum()) * 100
)

# Exibir o resultado
print(top_3_emotions.head())

              id      emotion  probability_normalized
0       $5_a_day  disapproval               38.706005
1       $5_a_day   admiration               33.133632
2       $5_a_day    annoyance               28.160362
3  009_re_cyborg  disapproval               36.023280
4  009_re_cyborg    annoyance               32.165396


In [17]:
top_3_emotions

,id,emotion,probability_normalized
0,$5_a_day,disapproval,38.706005
1,$5_a_day,admiration,33.133632
2,$5_a_day,annoyance,28.160362
3,009_re_cyborg,disapproval,36.023280
4,009_re_cyborg,annoyance,32.165396
...,...,...,...
202807,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,approval,38.438228
202808,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,fear,22.178533
202809,zz_top_that_little_ol_band_from_texas,amusement,35.673589
202810,zz_top_that_little_ol_band_from_texas,joy,34.614795


In [18]:
# Group by 'id' and create a list of tuples (emotion, probability_normalized)
new_df = top_3_emotions.groupby('id').apply(
    lambda x: list(zip(x['emotion'], x['probability_normalized'])))
new_df = new_df.reset_index()
new_df.columns = ['id', 'emotions']

new_df

,id,emotions
0,$5_a_day,"[(disapproval, 38.70600543243529), (admiration, 33.1336321492672), (annoyance, 28.16036241829752)]"
1,009_re_cyborg,"[(disapproval, 36.023279538211014), (annoyance, 32.165396240044096), (confusion, 31.81132422174489)]"
2,00_mhz,"[(annoyance, 41.840924718059824), (admiration, 36.49896509094363), (joy, 21.660110190996555)]"
3,1,"[(admiration, 61.524699999999996), (approval, 35.5203), (love, 2.955)]"
4,1-day,"[(admiration, 41.199610661687295), (amusement, 32.26985631655974), (disappointment, 26.530533021752966)]"
...,...,...
67599,zus_and_zo_2003,"[(desire, 40.01421221408237), (amusement, 30.742439443586793), (joy, 29.243348342330837)]"
67600,zvenigora,"[(confusion, 71.48010000000001), (disapproval, 17.093200000000003), (annoyance, 11.426700000000002)]"
67601,zwei_mutter_2013,"[(approval, 47.60089193324174), (admiration, 30.408156407361503), (realization, 21.990951659396757)]"
67602,zycie_jako_smiertelna_choroba_przenoszona_droga_plciowa_2000,"[(sadness, 39.38323877121109), (approval, 38.43822776765831), (fear, 22.178533461130584)]"


In [19]:
# Merge movies and new_df

merged_df = movies.merge(new_df, on='id', how='left')

merged_df

,id,title,audienceScore,tomatoMeter,runtimeMinutes,originalLanguage,director,writer,release_year,genre,emotions
0,love_lies,"Love, Lies",43.00,65.76,120.0,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,[Drama],"[(admiration, 78.20800919346894), (joy, 10.896268781721222), (approval, 10.89572202480984)]"
1,adrift_2018,Adrift,65.00,69.00,120.0,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018,"[Adventure, Drama, Romance]","[(admiration, 42.079146438693414), (disappointment, 29.810255887131987), (approval, 28.1105976741746)]"
2,1035316-born_to_kill,Born to Kill,74.00,83.00,92.0,English,Robert Wise,"Eve Greene,Richard Macaulay",1947,"[Crime, Drama]","[(admiration, 41.51550238655111), (disgust, 38.18461517417403), (disappointment, 20.299882439274857)]"
3,garden_murder_case,The Garden Murder Case,55.67,65.76,61.0,English,Edwin L. Marin,Unknown,2016,[Thriller],"[(amusement, 47.11795941712488), (admiration, 30.561675917621006), (approval, 22.320364665254107)]"
4,margarita_happy_hour,Margarita Happy Hour,55.67,76.00,98.0,English,Ilya Chaiken,Ilya Chaiken,2002,[Drama],"[(desire, 45.54859553528897), (admiration, 29.028624793560592), (joy, 25.42277967115044)]"
...,...,...,...,...,...,...,...,...,...,...,...
67599,operation_goldenshell,Operation Goldenshell (Operación Concha),55.67,65.76,88.0,Unknown,Antonio Cuadri,Unknown,Unknown,[Unknown],"[(amusement, 66.48219999999999), (realization, 20.131499999999996), (approval, 13.386299999999999)]"
67600,stag_night_of_the_dead,Stag Night of the Dead,43.00,65.76,81.0,English,Neil Jones,Neil Jones,2016,"[Horror, Action, Comedy]","[(approval, 60.283699999999996), (realization, 27.470399999999994), (admiration, 12.245899999999999)]"
67601,fun_size,Fun Size,47.00,25.00,86.0,English,Josh Schwartz,Max Werner,2012,"[Holiday, Comedy]","[(confusion, 48.71943200404655), (disappointment, 26.36692449980985), (amusement, 24.913643496143578)]"
67602,dassehra,Dassehra,55.67,65.76,131.0,Hindi,Manish Vatsalya,Saurabh Choudhary,2019,"[Action, Thriller]","[(approval, 71.40657548299487), (admiration, 20.783216447748366), (annoyance, 7.810208069256784)]"


In [20]:
merged_df.isna().sum()

id                  0
title               0
audienceScore       0
tomatoMeter         0
runtimeMinutes      0
originalLanguage    0
director            0
writer              0
release_year        0
genre               0
emotions            0
dtype: int64

In [21]:
merged_df.to_csv("movies_emotions.csv", index=False)

In [22]:
movies = pd.read_csv('/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/movies_emotions.csv')

In [23]:
movies.head()

,id,title,audienceScore,tomatoMeter,runtimeMinutes,originalLanguage,director,writer,release_year,genre,emotions
0,love_lies,"Love, Lies",43.00,65.76,120.0,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,['Drama'],"[('admiration', 78.20800919346894), ('joy', 10.896268781721222), ('approval', 10.89572202480984)]"
1,adrift_2018,Adrift,65.00,69.00,120.0,English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",2018,"['Adventure', 'Drama', 'Romance']","[('admiration', 42.079146438693414), ('disappointment', 29.810255887131987), ('approval', 28.1105976741746)]"
2,1035316-born_to_kill,Born to Kill,74.00,83.00,92.0,English,Robert Wise,"Eve Greene,Richard Macaulay",1947,"['Crime', 'Drama']","[('admiration', 41.51550238655111), ('disgust', 38.18461517417403), ('disappointment', 20.299882439274857)]"
3,garden_murder_case,The Garden Murder Case,55.67,65.76,61.0,English,Edwin L. Marin,Unknown,2016,['Thriller'],"[('amusement', 47.11795941712488), ('admiration', 30.561675917621006), ('approval', 22.320364665254107)]"
4,margarita_happy_hour,Margarita Happy Hour,55.67,76.00,98.0,English,Ilya Chaiken,Ilya Chaiken,2002,['Drama'],"[('desire', 45.54859553528897), ('admiration', 29.028624793560592), ('joy', 25.42277967115044)]"


#### Normalizing movie titles

In [24]:
# Function to normalize names
def normalize_name(name):
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Remove special characters
    name = name.lower().strip()  # Convert to lowercase and remove extra spaces
    return name

In [25]:
# Creating normalized title colunm in the movies df

movies['title_normalized'] = movies['title'].apply(normalize_name)

Delete duplicated normalized titles:

In [26]:
# Step 1: Group by 'id' and create the 'count' column
# Group by 'id' and count occurrences in the reviews table
reviews_count = reviews['id'].value_counts().reset_index()
reviews_count.columns = ['id', 'count']

# Add the 'count' column to the movies DataFrame
movies = movies.merge(reviews_count, on='id', how='left')

# Fill NaN values with 0 (movies with no reviews)
movies['count'] = movies['count'].fillna(0)

# Step 2: Check for duplicate titles
# Create a DataFrame with one row per movie (using the first occurrence of each 'id')
movies_unique = movies.drop_duplicates(subset=['id'], keep='first')

# Check for duplicate normalized titles
duplicates = movies_unique[movies_unique['title_normalized'].duplicated(keep=False)]
print("Duplicate titles (considering one entry per movie):")
print(duplicates[['id', 'title_normalized']].sort_values(by='title_normalized'))

# Step 3: Remove movies with fewer occurrences
# Sort movies by normalized title and review count
movies_sorted = movies.sort_values(by=['title_normalized', 'count'], ascending=[True, False])

# Keep only the first occurrence of each normalized title (the one with the most reviews)
movies_no_duplicates = movies_sorted.drop_duplicates(subset=['title_normalized'], keep='first')

# Check if there are still duplicates
if movies_no_duplicates['title_normalized'].duplicated().any():
    print("There are still duplicate titles after removal.")
else:
    print("All duplicate titles have been removed.")

# Display the final DataFrame
print("\nFinal DataFrame without duplicates:")
print(movies_no_duplicates)

Duplicate titles (considering one entry per movie):
                       id title_normalized
15730              1_2013                1
2681   1-one-human-minute                1
66029              1_2017                1
964                     1                1
50302                 ten               10
...                   ...              ...
58403            zoo_2018              zoo
15094           zoom_2016             zoom
53027           zoom_2006             zoom
40729                zulu             zulu
52700           zulu_2013             zulu

[9566 rows x 2 columns]
All duplicate titles have been removed.

Final DataFrame without duplicates:
                                                                 id  \
49201                                          my_missing_valentine   
53378                                                        00_mhz   
16516                                                 009_re_cyborg   
22292                                         

In [27]:
movies = movies_no_duplicates

In [28]:
movies.head()

,id,title,audienceScore,tomatoMeter,runtimeMinutes,originalLanguage,director,writer,release_year,genre,emotions,title_normalized,count
49201,my_missing_valentine,消失的情人節,55.67,65.76,119.0,Chinese,Yu-Hsun Chen,"Yu-Hsun Chen,Yu-Hsun Chen",Unknown,"['Comedy', 'Drama', 'Fantasy', 'Foreign', 'Romance']","[('admiration', 52.16256606223262), ('excitement', 24.98463494276986), ('joy', 22.852798994997528)]",,3
53378,00_mhz,0.0 MHz,33.00,65.76,101.0,Korean,Yoo Sun-Dong,Jang Jak,2020,['Horror'],"[('annoyance', 41.840924718059824), ('admiration', 36.49896509094363), ('joy', 21.660110190996555)]",00 mhz,3
16516,009_re_cyborg,009 Re: Cyborg,43.00,23.00,103.0,Japanese,Kenji Kamiyama,Kenji Kamiyama,2015,"['Action', 'Sci-fi', 'Animation']","[('disapproval', 36.023279538211014), ('annoyance', 32.165396240044096), ('confusion', 31.81132422174489)]",009 re cyborg,13
22292,45,0.45,38.00,65.76,101.0,English,Gary Lennon,Gary Lennon,2020,"['Crime', 'Drama', 'Thriller']","[('annoyance', 80.9424), ('anger', 14.1899), ('approval', 4.8677)]",045,1
66029,1_2017,1%,82.00,47.00,88.0,Unknown,Stephen McCallum,Unknown,Unknown,['Unknown'],"[('admiration', 40.5276835242493), ('disappointment', 30.096030871256218), ('approval', 29.376285604494484)]",1,17


In [29]:
movies.shape

(61999, 13)

In [30]:
movies[movies['id'] == '$5_a_day']

,id,title,audienceScore,tomatoMeter,runtimeMinutes,originalLanguage,director,writer,release_year,genre,emotions,title_normalized,count
60400,$5_a_day,Five Dollars a Day,49.0,65.76,98.0,English,Nigel Cole,"Neal H. Dobrofsky,Tippi Dobrofsky",2010,['Comedy'],"[('disapproval', 38.70600543243529), ('admiration', 33.1336321492672), ('annoyance', 28.16036241829752)]",five dollars a day,4


In [31]:
movies.to_csv("movies_final.csv", index=False)

#### Transforming df into wide format

In [32]:
# Transforma df into wide format
reviews_wide = top_3_emotions.pivot(index='id', columns='emotion', values='probability_normalized')

# Fill missing values with 0
reviews_wide = reviews_wide.fillna(0)

reviews_wide = reviews_wide.round(3)  

reviews_wide.head()

emotion,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
id,,,,,,,,,,,,,,,,,,,,,
$5_a_day,33.134,0.00,0.0,28.160,0.00,0.0,0.000,0.0,0.0,0.000,...,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009_re_cyborg,0.000,0.00,0.0,32.165,0.00,0.0,31.811,0.0,0.0,0.000,...,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00_mhz,36.499,0.00,0.0,41.841,0.00,0.0,0.000,0.0,0.0,0.000,...,21.66,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,61.525,0.00,0.0,0.000,35.52,0.0,0.000,0.0,0.0,0.000,...,0.00,2.955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-day,41.200,32.27,0.0,0.000,0.00,0.0,0.000,0.0,0.0,26.531,...,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
reviews_wide.shape

(67604, 27)

#### Removing IDs that are not present in the movies DataFrame from the reviews_wide DataFrame

In [34]:
reviews_wide = reviews_wide.loc[reviews_wide.index.isin(movies['id'])]

print(reviews_wide.shape)

reviews_wide.head()

(61999, 27)


emotion,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
id,,,,,,,,,,,,,,,,,,,,,
$5_a_day,33.134,0.00,0.0,28.160,0.0,0.0,0.000,0.0,0.0,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009_re_cyborg,0.000,0.00,0.0,32.165,0.0,0.0,31.811,0.0,0.0,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00_mhz,36.499,0.00,0.0,41.841,0.0,0.0,0.000,0.0,0.0,0.000,...,21.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-day,41.200,32.27,0.0,0.000,0.0,0.0,0.000,0.0,0.0,26.531,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-violent-women,0.000,0.00,0.0,0.000,0.0,0.0,0.000,0.0,0.0,60.545,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Similarity Calculation

Using PCA (Principal Component Analysis) to reduce data dimensionality before calculating similarity:

In [35]:
# Function to calculate similarity matrix based on emotion data
def calculate_similarity_matrix(reviews_wide):
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=0.97)  # Keep 97% of the variance
    reviews_reduced = pca.fit_transform(reviews_wide.values)
    
    # Calculate cosine similarity
    sim_matrix = cosine_similarity(reviews_reduced)
    
    # Convert to DataFrame
    sim_df = pd.DataFrame(sim_matrix, index=reviews_wide.index, columns=reviews_wide.index)
    
    return sim_df

In [ ]:
sim_df = calculate_similarity_matrix(reviews_wide)

In [ ]:
sim_df.head(10)

In [ ]:
size_bytes = sys.getsizeof(sim_df)

size_gb = size_bytes / 1e9

print(f"Size of the matrix: {size_gb:.6f} GB")

# Converting sim_df Into a Smaller df

#### Testing with a sample df

In [ ]:
sample_df = sim_df.head(10)

In [40]:
# sample_df.to_csv("sample_df.csv", index=False)

In [ ]:
sample_df.dtypes[0]

In [34]:
movies_subset = movies[['id', 'genre']].copy()
movies_subset.columns

Index(['id', 'genre'], dtype='object')

In [94]:
# Directory to save processed chunks
output_dir = '/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks_sample'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Function to check if two movies share at least one genre
def has_common_genre(genres1, genres2):
    # If one of the genres is 'Unknown', return True (consider any genre)
    if 'Unknown' in genres1 or 'Unknown' in genres2:
        return True
    # Otherwise, check if there are common genres
    return not set(genres1).isdisjoint(genres2)

# Function to process each chunk
def process_chunk(chunk, movies):
    # Reset the index to make 'id' a column
    chunk = chunk.reset_index()
    
    # Step 2: Melt the matrix to transform it into pairs of IDs and similarity scores
    chunk_melt = chunk.melt(id_vars='id', var_name='id2', value_name='score')
    chunk_melt.rename(columns={'id': 'id1'}, inplace=True)
    
    # Convert columns to lighter data types to save memory
    chunk_melt['id1'] = chunk_melt['id1'].astype('category')
    chunk_melt['id2'] = chunk_melt['id2'].astype('category')
    chunk_melt['score'] = chunk_melt['score'].astype('float32')
    
    # Debug: Check the result of melt
    print("After melt:")
    print(chunk_melt.head())
    
    # Step 3: Inner join with the genres table to filter movies with at least one common genre
    chunk_melt = chunk_melt.merge(movies_subset, left_on='id1', right_on='id', suffixes=('', '_1'))
    chunk_melt = chunk_melt.merge(movies_subset, left_on='id2', right_on='id', suffixes=('', '_2'))
    
    # Debug: Check the result of merging
    print("After merging with genres:")
    print(chunk_melt.head())
    
    # Filter movies that share at least one genre (or have 'Unknown')
    chunk_melt = chunk_melt[chunk_melt.apply(lambda row: has_common_genre(row['genre'], row['genre_2']), axis=1)]
    
    # Step 4: Group by id1 and keep the top 6 most similar movies
    chunk_melt = chunk_melt.sort_values(by=['id1', 'score'], ascending=[True, False])
    chunk_melt = chunk_melt.groupby('id1').head(6)
    
    # Exclude the movie itself (where id1 == id2)
    chunk_melt = chunk_melt[chunk_melt['id1'] != chunk_melt['id2']]
    
    # Keep only the top 5 similar movies
    chunk_melt = chunk_melt.groupby('id1').head(5)
    
    # Return relevant columns, including genres
    return chunk_melt[['id1', 'id2', 'score', 'genre', 'genre_2']]

# Step 1: Split the sample DataFrame into chunks (e.g., 2 rows per chunk)
chunk_size = 2  # Small chunk size for testing
num_chunks = int(np.ceil(sample_df.shape[0] / chunk_size))

# List to store paths of processed files
processed_files = []

# Process each chunk
for i in range(num_chunks):
    # Check if the chunk has already been processed
    file_path = os.path.join(output_dir, f'chunk_{i}.parquet')
    if os.path.exists(file_path):
        print(f"Chunk {i} already processed. Skipping...")
        processed_files.append(file_path)
        continue
    
    print(f"Processing chunk {i + 1}/{num_chunks}")
    start = i * chunk_size
    end = min((i + 1) * chunk_size, sample_df.shape[0])
    chunk = sample_df.iloc[start:end, :]  # Select the current chunk
    
    # Process the chunk
    result_chunk = process_chunk(chunk, movies)
    
    # Save the processed chunk to a Parquet file
    result_chunk.to_parquet(file_path)
    processed_files.append(file_path)
    print(f"Chunk {i} saved to {file_path}")

# Step 5: Concatenate all processed chunks
print("Concatenating all processed chunks...")
df_final_sample = pd.concat([pd.read_parquet(file, columns=['id1', 'id2', 'score', 'genre', 'genre_2']) for file in processed_files], ignore_index=True)

# Display the final DataFrame
print("Final DataFrame:")
print(df_final_sample.head())

# Save the final DataFrame (optional)
df_final_sample.to_parquet(os.path.join(output_dir, 'final_result.parquet'))
print("Process completed!")

Processing chunk 1/5
After melt:
             id1            id2     score
0       $5_a_day       $5_a_day  1.000000
1  009_re_cyborg       $5_a_day  0.642029
2       $5_a_day  009_re_cyborg  0.642029
3  009_re_cyborg  009_re_cyborg  1.000000
4       $5_a_day         00_mhz  0.548673
After merging with genres:
             id1            id2     score             id  \
0       $5_a_day       $5_a_day  1.000000       $5_a_day   
1  009_re_cyborg       $5_a_day  0.642029  009_re_cyborg   
2       $5_a_day  009_re_cyborg  0.642029       $5_a_day   
3  009_re_cyborg  009_re_cyborg  1.000000  009_re_cyborg   
4       $5_a_day         00_mhz  0.548673       $5_a_day   

                         genre           id_2                      genre_2  
0                     [Comedy]       $5_a_day                     [Comedy]  
1  [Action, Sci-fi, Animation]       $5_a_day                     [Comedy]  
2                     [Comedy]  009_re_cyborg  [Action, Sci-fi, Animation]  
3  [Action, Sci-fi,

In [96]:
pd.set_option('display.max_rows', None) 

In [98]:
df_final_sample

,id1,id2,score,genre,genre_2
0,$5_a_day,hotel_de_love,0.995643,[Comedy],"[Romance, Comedy]"
1,$5_a_day,the_midnight_gang,0.993423,[Comedy],[Unknown]
2,$5_a_day,ce_quil_ne_faut_pas_dire,0.989711,[Comedy],[Unknown]
3,$5_a_day,simple_men,0.986981,[Comedy],"[Comedy, Drama, Romance]"
4,$5_a_day,handsome_a_netflix_mystery_movie,0.986540,[Comedy],[Comedy]
5,009_re_cyborg,the_lion_of_judah,0.990261,"[Action, Sci-fi, Animation]","[Family, Holiday, Drama, Animation]"
6,009_re_cyborg,gog,0.974672,"[Action, Sci-fi, Animation]",[Sci-fi]
7,009_re_cyborg,hinda_and_her_sisterrrz,0.952611,"[Action, Sci-fi, Animation]",[Unknown]
8,009_re_cyborg,william_s_burroughs_commissioner_of_sewers,0.934580,"[Action, Sci-fi, Animation]",[Unknown]
9,009_re_cyborg,malicedoll,0.920395,"[Action, Sci-fi, Animation]",[Unknown]


In [36]:
# Convert 'genre' from str into list
movies['genre'] = movies['genre'].apply(ast.literal_eval)
movies_subset['genre'] = movies_subset['genre'].apply(ast.literal_eval)

/var/folders/8f/yz06rsgd4g58pv0x59d7qyc40000gn/T/ipykernel_84867/993239469.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genre'] = movies['genre'].apply(ast.literal_eval)


In [1]:
# Check the 'genre' type after conversion
print("Type of 'genre' in movies after conversion:", type(movies['genre'].iloc[0]))
print("Type of 'genre' in movies_subset after conversion:", type(movies_subset['genre'].iloc[0]))

NameError: name 'movies' is not defined

#### Applying it to the real sim_df

In [108]:
# Directory to save processed chunks
output_dir = '/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Function to check if two movies share at least one genre
def has_common_genre(genres1, genres2):

    if 'Unknown' in genres1 or 'Unknown' in genres2:
        return True

    return not set(genres1).isdisjoint(genres2)

# Function to process each chunk
def process_chunk(chunk, movies):
    # Reset the index to make 'id' a column
    chunk = chunk.reset_index()
    
    # Melt the matrix to transform it into pairs of IDs and similarity scores
    chunk_melt = chunk.melt(id_vars='id', var_name='id2', value_name='score')
    chunk_melt.rename(columns={'id': 'id1'}, inplace=True)
    
    # Convert columns to lighter data types to save memory
    chunk_melt['id1'] = chunk_melt['id1'].astype('category')
    chunk_melt['id2'] = chunk_melt['id2'].astype('category')
    chunk_melt['score'] = chunk_melt['score'].astype('float32')
    
    # Inner join with the genres table to filter movies with at least one common genre
    chunk_melt = chunk_melt.merge(movies_subset, left_on='id1', right_on='id', suffixes=('', '_1'))
    chunk_melt = chunk_melt.merge(movies_subset, left_on='id2', right_on='id', suffixes=('', '_2'))
    
    # Rename gender columns
    chunk_melt.rename(columns={'genre': 'genre', 'genre_1': 'genre_2'}, inplace=True)
    
    # Check types after merge
    print("Type of 'genre' after merge:", type(chunk_melt['genre'].iloc[0]))
    print("Type of 'genre_2' after merge:", type(chunk_melt['genre_2'].iloc[0]))
    
    # Filter out movies that don't share a common genre
    chunk_melt = chunk_melt[chunk_melt.apply(lambda row: has_common_genre(row['genre'], row['genre_2']), axis=1)]
   
    # Group by id1 and keep the top 6 most similar movies
    chunk_melt = chunk_melt.sort_values(by=['id1', 'score'], ascending=[True, False])
    chunk_melt = chunk_melt.groupby('id1').head(6)
    
    chunk_melt = chunk_melt[chunk_melt['id1'] != chunk_melt['id2']]
    
    chunk_melt = chunk_melt.groupby('id1').head(5)
    
    return chunk_melt[['id1', 'id2', 'score', 'genre', 'genre_2']]

# Split the DataFrame into chunks
chunk_size = 300  
num_chunks = int(np.ceil(sim_df.shape[0] / chunk_size))

# List to store paths of processed files
processed_files = []

# Process each chunk
for i in range(num_chunks):
    # Check if the chunk has already been processed
    file_path = os.path.join(output_dir, f'chunk_{i}.parquet')
    if os.path.exists(file_path):
        print(f"Chunk {i} already processed. Skipping...")
        processed_files.append(file_path)
        continue
    
    print(f"Processing chunk {i + 1}/{num_chunks}")
    start = i * chunk_size
    end = min((i + 1) * chunk_size, sim_df.shape[0])
    chunk = sim_df.iloc[start:end, :]  # Select the current chunk
    
    # Process the chunk
    result_chunk = process_chunk(chunk, movies_subset)
    
    # Save the processed chunk to a Parquet file
    result_chunk.to_parquet(file_path)
    processed_files.append(file_path)
    print(f"Chunk {i} saved to {file_path}")

# Concatenate all processed chunks
print("Concatenating all processed chunks...")
df_final = pd.concat([pd.read_parquet(file, columns=['id1', 'id2', 'score', 'genre', 'genre_2']) for file in processed_files], ignore_index=True)

# Display the final DataFrame
print("Final DataFrame:")
print(df_final.head())

# Save the final DataFrame
df_final.to_parquet(os.path.join(output_dir, 'df_final.parquet'))
print("Process completed!")

Processing chunk 1/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 0 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_0.parquet
Processing chunk 2/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 1 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_1.parquet
Processing chunk 3/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 2 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_2.parquet
Processing chunk 4/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 3 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_3.parquet
Processing chunk 5/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 4 saved to /Users

Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 38 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_38.parquet
Processing chunk 40/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 39 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_39.parquet
Processing chunk 41/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 40 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_40.parquet
Processing chunk 42/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 41 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_41.parquet
Processing chunk 43/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 42 saved to /Users/danielebe

Chunk 75 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_75.parquet
Processing chunk 77/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 76 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_76.parquet
Processing chunk 78/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 77 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_77.parquet
Processing chunk 79/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 78 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_78.parquet
Processing chunk 80/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 79 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_79.parquet
Processing chunk 81

Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 113 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_113.parquet
Processing chunk 115/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 114 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_114.parquet
Processing chunk 116/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 115 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_115.parquet
Processing chunk 117/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 116 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_116.parquet
Processing chunk 118/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 117 saved to /Us

Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 150 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_150.parquet
Processing chunk 152/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 151 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_151.parquet
Processing chunk 153/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 152 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_152.parquet
Processing chunk 154/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 153 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_153.parquet
Processing chunk 155/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 154 saved to /Us

Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 187 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_187.parquet
Processing chunk 189/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 188 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_188.parquet
Processing chunk 190/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 189 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_189.parquet
Processing chunk 191/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 190 saved to /Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks/chunk_190.parquet
Processing chunk 192/207
Type of 'genre' after merge: <class 'list'>
Type of 'genre_2' after merge: <class 'list'>
Chunk 191 saved to /Us

In [3]:
size_bytes = sys.getsizeof(df_final)

size_gb = size_bytes / 1e9

print(f"Size of the matrix: {size_gb:.6f} GB")

NameError: name 'sys' is not defined

In [4]:
df_final = dd.read_parquet('/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks').compute()

NameError: name 'dd' is not defined

In [43]:
df_final

,id1,id2,score,genre,genre_2
7426500,$5_a_day,hotel_de_love,0.995643,[Comedy],"[Romance, Comedy]"
15930600,$5_a_day,the_midnight_gang,0.993423,[Comedy],[Unknown]
3668400,$5_a_day,ce_quil_ne_faut_pas_dire,0.989711,[Comedy],[Unknown]
13512900,$5_a_day,simple_men,0.986981,[Comedy],"[Comedy, Drama, Romance]"
6933300,$5_a_day,handsome_a_netflix_mystery_movie,0.986540,[Comedy],[Comedy]
...,...,...,...,...,...
309990,zz_top_that_little_ol_band_from_texas,road_scholar,0.998716,"[Documentary, Music]",[Documentary]
309991,zz_top_that_little_ol_band_from_texas,yucatan_2018,0.998606,"[Documentary, Music]",[Unknown]
309992,zz_top_that_little_ol_band_from_texas,1212267-blast,0.995124,"[Documentary, Music]",[Documentary]
309993,zz_top_that_little_ol_band_from_texas,the_cigarette_girl_from_mosselprom,0.992749,"[Documentary, Music]",[Unknown]


In [ ]:
df_final_clean = df_final.drop_duplicates(subset=['id1', 'id2'])
df_final_clean.to_csv("df_final_clean.csv", index=False)

In [ ]:
df_final_clean = df_final.drop_duplicates(subset=['id1', 'id2'])

# Group by id1 and count unique id2 values
group_counts = df_final_clean.groupby('id1')['id2'].nunique()

# Check the distribution of the counts
print("Distribution of unique recommendations per film:")
print(group_counts.value_counts())

# List films that don't have exactly 5 unique recommendations
films_not_five = group_counts[group_counts != 5]
print("\nFilms with a number of recommendations different from 5:")
print(films_not_five)

In [ ]:
df_final = dd.read_parquet('/Users/danielebelmiro/Data_Analytics_Bootcamp/Rotten/processed_chunks').compute()

# Group by id1 and count unique id2 values
group_counts = df_final.groupby('id1')['id2'].nunique()

# Check the distribution of the counts
print("Distribution of unique recommendations per film:")
print(group_counts.value_counts())

# List films that don't have exactly 5 unique recommendations
films_not_five = group_counts[group_counts != 5]
print("\nFilms with a number of recommendations different from 5:")
print(films_not_five)

In [ ]:
df_final.shape

In [ ]:
df_final_clean.shape

# Validating the Similarity Calculation

In [38]:
colunas = ['approval', 'amusement', 'surprise']

condicao = (reviews_wide[colunas] > 20)

linhas_filtradas = reviews_wide[condicao.sum(axis=1) >= 3]

print(linhas_filtradas)

emotion                  admiration  amusement  anger  annoyance  approval  \
id                                                                           
10008698-king_corn              0.0     34.124    0.0        0.0    31.708   
1006250-dragnet                 0.0     33.403    0.0        0.0    28.819   
1038237-platinum_blonde         0.0     43.619    0.0        0.0    26.868   
1118698-empire                  0.0     30.693    0.0        0.0    28.538   
1120843-hunted                  0.0     36.116    0.0        0.0    31.508   
...                             ...        ...    ...        ...       ...   
used_people                     0.0     36.491    0.0        0.0    30.280   
web_junkie                      0.0     40.971    0.0        0.0    29.956   
wheres_marlowe                  0.0     33.271    0.0        0.0    20.512   
workforce                       0.0     32.062    0.0        0.0    26.473   
wrong_turn_2                    0.0     31.592    0.0        0.0

In [5]:
# Look at the matrix and compare the similarity between films you already know 
# are similar or different based on their emotion percentage profiles.
# If two films have very similar emotion distributions, the similarity score should be high.

# Films with similar emotion percentages
film_a = reviews_wide.loc['wrong_turn_2']
film_b = reviews_wide.loc['10008698-king_corn']

# Check their similarity in the matrix
similarity = sim_df.loc['wrong_turn_2', '10008698-king_corn']
print(f"Similarity: {similarity}")

NameError: name 'reviews_wide' is not defined

In [6]:
# Films with different emotion percentages
film_a = reviews_wide.loc['katherine_ryan_glitter_room']
film_b = reviews_wide.loc['009_re_cyborg']

# Check their similarity in the matrix
similarity = sim_df.loc['katherine_ryan_glitter_room', '009_re_cyborg']
print(f"Similarity: {similarity}")

NameError: name 'reviews_wide' is not defined

In [7]:
# Apply PCA to reduce dimensionality
pca = PCA(n_components=0.97)  # Retain 97% of variance
reviews_reduced = pca.fit_transform(reviews_wide.values)

# Choose two films to compare
film1_id = 'katherine_ryan_glitter_room'  # Replace with the first film's ID
film2_id = '009_re_cyborg'               # Replace with the second film's ID

# Find the films' indices in the reviews_wide matrix
film1_idx = reviews_wide.index.get_loc(film1_id)
film2_idx = reviews_wide.index.get_loc(film2_id)

# Select the film vectors after PCA
film1_vector = reviews_reduced[film1_idx].reshape(1, -1)  # Shape into (1, n_features)
film2_vector = reviews_reduced[film2_idx].reshape(1, -1)

# Compute cosine similarity manually
manual_similarity = cosine_similarity(film1_vector, film2_vector)[0][0]

# Retrieve the corresponding value from the sim_df matrix
matrix_similarity = sim_df.loc[film1_id, film2_id]

# Display results
print(f"Manual similarity between {film1_id} and {film2_id}: {manual_similarity}")
print(f"Similarity in sim_df: {matrix_similarity}")

# Check if the values match (or are very close, due to rounding)
if np.isclose(manual_similarity, matrix_similarity, atol=1e-6):
    print("Values are identical (within a small tolerance).")
else:
    print("Values differ.")

NameError: name 'PCA' is not defined

# Main Recommendation Function

In [49]:
def recommend_similar_movies(sim_df, movies, reviews, top_n=5):
    movies = movies.copy()

    if isinstance(movies['genre'].iloc[0], str):
        movies['genre'] = movies['genre'].apply(ast.literal_eval)

    while True:
        # Ask the user for their favorite movie
        favorite_movie = input("Enter the name of your favorite movie (or type 'exit' to quit): ").strip()

        # Allow the user to exit
        if favorite_movie.lower() == 'exit':
            print("Exiting the recommendation system. Goodbye!")
            return None

        # Normalize the movie name
        favorite_movie_normalized = normalize_name(favorite_movie)

        # Find the movie in the dataset
        matching_movies = movies[movies['title_normalized'] == favorite_movie_normalized]
        if matching_movies.empty:
            print(f"The movie '{favorite_movie}' was not found. Please check the name and try again.")
            continue  
        else:
            favorite_movie_id = matching_movies.iloc[0]['id']
            favorite_movie_title = matching_movies.iloc[0]['title']
            print(f"Found movie: {favorite_movie_title} (ID: {favorite_movie_id})")
            break  

    # Check if the movie is in the similarity matrix
    if favorite_movie_id not in sim_df.columns:
        print(f"Movie ID '{favorite_movie_id}' not found in the similarity matrix.")
        return None

    # Get similarity scores and sort by highest similarity
    movie_similarities = sim_df[favorite_movie_id].sort_values(ascending=False)

    # Remove the movie itself from recommendations
    movie_similarities = movie_similarities.drop(favorite_movie_id, errors='ignore')

    # Get genres of the favorite movie
    favorite_movie_genres = set(matching_movies['genre'].explode().values)
    print(favorite_movie_genres)

    # Filter recommendations by shared genre
    recommended_ids = movies[movies['genre'].apply(lambda genres: any(genre in favorite_movie_genres for genre in genres))]['id'].unique()
    movie_similarities = movie_similarities[movie_similarities.index.isin(recommended_ids)]

    # Get top N recommendations
    top_recommendations = movie_similarities.head(top_n).reset_index()
    top_recommendations.columns = ['id', 'similarity']

    # Merge with movie details
    recommended_movies = top_recommendations.merge(movies, on='id', how='left')

    # Select relevant columns
    result = recommended_movies[['id', 'title', 'director', 'originalLanguage', 'runtimeMinutes', 
                                 'genre', 'release_year', 'tomatoMeter', 'audienceScore', 
                                 'similarity', 'emotions']]

    # Display emotional profile of favorite movie
    favorite_movie_emotions = matching_movies.iloc[0]['emotions']
    print(f"Emotional profile of '{favorite_movie_title}':")  
    if isinstance(favorite_movie_emotions, list):
        print(f"   ❤️ Emotions: {', '.join([f'{mood} ({percentage:.1f}%)' for mood, percentage in favorite_movie_emotions])}")
    else:
        print(f"   ❤️ Emotions: {favorite_movie_emotions}")  
    print("-" * 50)
    
    
    # Print formatted recommendations
    print(f"\nTop {top_n} movie recommendations based on '{favorite_movie_title}':\n")


    # Display recommendations
    for _, row in result.iterrows():
        print(f"🎬 Movie: {row['title']}")
        print(f"   🎬 Director: {row['director']}")
        print(f"   🌍 Language: {row['originalLanguage']}")
        print(f"   ⏳ Duration: {row['runtimeMinutes']} min")
        print(f"   🎭 Genre: {', '.join(row['genre'])}")
        print(f"   📅 Year: {row['release_year']}")
        print(f"   🍅 Tomatometer: {row['tomatoMeter']}%")
        print(f"   🎟️ Audience Score: {row['audienceScore']}%")
        print(f"   🔗 Similarity Score: {row['similarity']:.5f}")
        if isinstance(row['emotions'], list):
            print(f"   ❤️ Emotions: {', '.join([f'{mood} ({percentage:.1f}%)' for mood, percentage in row['emotions']])}")
        else:
            print(f"   ❤️ Emotions: {row['emotions']}")  
        print("-" * 50)

    return result

# Generate Recommendations

In [50]:
recommendations = recommend_similar_movies(sim_df, movies, reviews, top_n=5)

Enter the name of your favorite movie (or type 'exit' to quit): five dollars a day
Found movie: Five Dollars a Day (ID: $5_a_day)
{'Comedy'}
Emotional profile of 'Five Dollars a Day':
   ❤️ Emotions: [('disapproval', 38.70600543243529), ('admiration', 33.1336321492672), ('annoyance', 28.16036241829752)]
--------------------------------------------------

Top 5 movie recommendations based on 'Five Dollars a Day':

🎬 Movie: Hotel de Love
   🎬 Director: Craig Rosenberg
   🌍 Language: English
   ⏳ Duration: 95.0 min
   🎭 Genre: Romance, Comedy
   📅 Year: Unknown
   🍅 Tomatometer: 22.0%
   🎟️ Audience Score: 54.0%
   🔗 Similarity Score: 0.99564
   ❤️ Emotions: [('disapproval', 37.67944043788483), ('admiration', 36.74494103061423), ('annoyance', 25.575618531500933)]
--------------------------------------------------
🎬 Movie: Simple Men
   🎬 Director: Hal Hartley
   🌍 Language: English
   ⏳ Duration: 105.0 min
   🎭 Genre: Comedy, Drama, Romance
   📅 Year: 2004
   🍅 Tomatometer: 91.0%
   🎟️ Au

In [51]:
recommendations

,id,title,director,originalLanguage,runtimeMinutes,genre,release_year,tomatoMeter,audienceScore,similarity,emotions
0,hotel_de_love,Hotel de Love,Craig Rosenberg,English,95.00,"[Romance, Comedy]",Unknown,22.00,54.0,0.995643,"[('disapproval', 37.67944043788483), ('admiration', 36.74494103061423), ('annoyance', 25.575618531500933)]"
1,simple_men,Simple Men,Hal Hartley,English,105.00,"[Comedy, Drama, Romance]",2004,91.00,89.0,0.986981,"[('disapproval', 44.171436068251964), ('admiration', 33.65403737437839), ('annoyance', 22.174526557369653)]"
2,handsome_a_netflix_mystery_movie,Handsome: A Netflix Mystery Movie,Jeff Garlin,English,103.00,[Comedy],2017,65.76,31.0,0.986540,"[('disapproval', 45.653808792131116), ('admiration', 30.554937861873288), ('annoyance', 23.791253345995585)]"
3,i_love_melvin,I Love Melvin,Don Weis,English,76.00,"[Musical, Comedy]",1953,65.76,58.0,0.986439,"[('admiration', 38.62660210233127), ('disapproval', 32.85559726258748), ('annoyance', 28.517800635081255)]"
4,everything_under_control,Everything Under Control,Unknown,Chinese,93.75,"[Comedy, Drama, Action, Horror]",2023,65.76,60.0,0.975437,"[('admiration', 39.48153287055353), ('disapproval', 30.333672560212143), ('annoyance', 30.18479456923434)]"
